# Сессия 2 – Оценка RAG с помощью ragas

Оцените минимальный конвейер RAG, используя метрики ragas: релевантность ответа, достоверность, точность контекста.


# Сценарий
Этот сценарий оценивает минимальный конвейер Retrieval Augmented Generation (RAG) локально. Мы:
- Определяем небольшой синтетический корпус документов.
- Встраиваем документы и реализуем простой поисковик по схожести.
- Генерируем обоснованные ответы с использованием локальной модели (Foundry Local / совместимой с OpenAI).
- Вычисляем метрики ragas (`answer_relevancy`, `faithfulness`, `context_precision`).
- Поддерживаем РЕЖИМ БЫСТРОГО ЗАПУСКА (переменная окружения `RAG_FAST=1`) для вычисления только релевантности ответа для быстрой итерации.

Используйте этот ноутбук, чтобы убедиться, что ваша локальная модель + стек встраивания генерируют фактически обоснованные ответы перед масштабированием на более крупные корпуса.


### Объяснение: Установка зависимостей
Устанавливаются необходимые библиотеки:
- `foundry-local-sdk` для управления локальными моделями.
- Клиентский интерфейс `openai`.
- `sentence-transformers` для плотных эмбеддингов.
- `ragas` + `datasets` для оценки и вычисления метрик.
- Адаптер `langchain-openai` для интерфейса LLM в ragas.

Можно запускать повторно; пропустите, если среда уже подготовлена.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### Объяснение: Основные импорты и метрики
Загружаются основные библиотеки и метрики раг. Ключевые элементы:
- SentenceTransformer для создания эмбеддингов.
- `evaluate` + выбранные метрики раг.
- `Dataset` для создания корпуса для оценки.
Эти импорты не инициируют удалённые вызовы (за исключением возможной загрузки модели из кэша для эмбеддингов).


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### Объяснение: Игрушечный корпус и эталонные ответы для вопросов
Определяет небольшой корпус в памяти (`DOCS`), набор пользовательских вопросов и ожидаемые эталонные ответы. Это позволяет быстро и детерминированно вычислять метрики без обращения к внешним данным. В реальных сценариях вы бы использовали выборку производственных запросов и тщательно подобранные ответы.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### Объяснение: Инициализация сервиса, эмбеддинги и защитный патч
Инициализирует локальный менеджер Foundry, применяет защитный патч от отклонений схемы для `promptTemplate`, определяет идентификатор модели, создает клиент, совместимый с OpenAI, и предварительно вычисляет плотные эмбеддинги для корпуса документов. Это создает повторно используемое состояние для извлечения и генерации.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### Объяснение: Функция Retriever
Определяет простую функцию поиска по векторному сходству, используя скалярное произведение нормализованных эмбеддингов. Возвращает топ-k документов (по умолчанию k=2). В производственной среде замените на индекс ANN (FAISS, Chroma, Milvus) для масштабируемости и снижения задержки.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### Объяснение: Функция генерации
`generate` создает ограниченный запрос (система инструктирует использовать ТОЛЬКО контекст) и вызывает локальную модель. Низкая температура (0.1) способствует точному извлечению информации, а не творческому подходу. Возвращает обрезанный текст ответа.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### Объяснение: Инициализация резервного клиента
Гарантирует, что `client` существует, даже если предыдущая ячейка инициализации была пропущена или завершилась ошибкой — предотвращает возникновение NameError на последующих этапах выполнения.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### Объяснение: Цикл оценки и метрики
Создает набор данных для оценки (обязательные столбцы: question, answer, contexts, ground_truths, reference), а затем выполняет итерацию по выбранным метрикам ragas.

Оптимизация:
- FAST_MODE ограничивается проверкой релевантности ответа для быстрых тестов.
- Цикл по каждой метрике позволяет избежать полной перерасчета, если одна из метрик не удалась.

Возвращает словарь метрика -> оценка (NaN в случае ошибки).


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
